In [6]:
import sys
import pandas as pd
import numpy as np
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from ckiptagger import construct_dictionary
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

host = 4
# host = 0

if host == 0:
    path = r'/Users/aron/Documents/GitHub/Perfume/2_NLP'
    dict_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/dict.xlsx'
    data_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/data.xls'
    
elif host == 4:
    path = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/'
    dict_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/dict.xlsx'
    data_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/data.xls'

    
# Codebase ......
path_codebase = [r'/Users/aron/Documents/GitHub/Arsenal/',
                 r'/home/aronhack/stock_predict/Function',
                 r'D:\GitHub\Arsenal',
                 r'D:\Data_Mining\Projects\Codebase_YZ',
                 r'/home/jupyter/Arsenal/20220522',
                 path + '/Function']


for i in path_codebase:    
    if i not in sys.path:
        sys.path = [i] + sys.path
    

import codebase_yz as cbyz
    
    
path_resource = path + '/Resource'
path_export = path + '/Export'

## Import Custom Dictionary

In [3]:
dict_df = pd.read_excel(dict_file)    
dict_li = dict_df['word'].tolist()
custom_dict = dict((el, 1) for el in dict_li)
custom_dict = construct_dictionary(custom_dict)

In [4]:
# 2. Load model
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
# https://drive.google.com/drive/folders/105IKCb88evUyLKlLondvDBoh7Dy_I1tm

ws = WS(path_resource + "/data")
pos = POS(path_resource + "/data")
ner = NER(path_resource + "/data")

/opt/anaconda3/lib/python3.7/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/opt/anaconda3/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/opt/anaconda3/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
/opt/anaconda3/lib/python3.7/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future ver

## 4. Run the WS-POS-NER pipeline

In [76]:
data = pd.read_excel(data_file)
data = data.dropna(subset=['title'], axis=0)
data = data[~data['title'].str.contains('廣告')]

# data = data[~data['name'].isna()]
# data = data[data.index<=300]

data = data.dropna(subset=['title'], axis=0)
sentence_list = data['title'].tolist()

# y = data['name'].tolist()
y = data['brand'].tolist()

In [78]:
data['title'] = data['title'].str.contains('找相似', '')

In [79]:
# Test Manually
data = data[data['title'].str.contains(' 萬寶龍星際旅者男性淡香水')]
sentence_list = data['title'].tolist()

In [30]:
word_sentence_list = ws(
    sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = custom_dict, # words in this dictionary are forced
)

pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [ ]:
df = pd.DataFrame()
for i, sentence in enumerate(sentence_list):
    
    new_li = list(zip(word_sentence_list[i],  pos_sentence_list[i]))
    cur_y = y[i]
    
    temp_df = pd.DataFrame(new_li, columns=['word', 'pos'])
    temp_df['sentence'] = i
    temp_df['tag'] = np.where(temp_df['word']==cur_y, '1', '0')
    
    df = df.append(temp_df)
    
df = df[['sentence', 'word', 'pos', 'tag']]
df

In [ ]:
df[df['tag']=='1']

In [ ]:
# 6. Show Results
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return


In [ ]:
# word_sentence_list[0]
# pos_sentence_list[0]

df[df['tag']=='1']

In [ ]:
for i, sentence in enumerate(sentence_list):
#     print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)

## Feature Extraction

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['word'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['tag'].values.tolist())]
        self.grouped = self.data.groupby('sentence').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

        
getter = SentenceGetter(df)
sentences = getter.sentences

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
### Split train and test sets
X = [sent2features(s) for s in sentences]
new_y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=0)

### Train a CRF model

In [ ]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [ ]:
trainer.set_params({
#     'c1': 1.0,   # coefficient for L1 penalty
#     'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [ ]:
trainer.params()

In [ ]:
trainer.train(path_export + '/ckip_tagger_model.crfsuite')

In [ ]:
trainer.logparser.last_iteration

In [ ]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open(path_export + '/ckip_tagger_model.crfsuite')

Let's tag a sentence to see how it works:

In [ ]:
y_pred = []
for i in range(len(y_test)):
    new_pred = tagger.tag(X_test[i])
    y_pred.append(new_pred)
#     example_sent = X_test[i]
#     print("Predicted:", ' '.join(tagger.tag(X_test[i])))    

In [ ]:
y_pred = cbyz.li_flatten(y_pred)
y_test = cbyz.li_flatten(y_test)

In [ ]:
# len(y_pred)
len(X_test)

In [ ]:
y_test[i for i in y_test]

In [ ]:
print(classification_report(y_pred=y_pred, y_true=y_test))
# print(classification_report(y_pred=y_pred, y_true=y_test, labels=new_classes))

## Let's check what classifier learned

In [ ]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

## Word2Vec

In [33]:
import gensim
from gensim import matutils
from gensim.models.word2vec import Word2Vec


model = gensim.models.KeyedVectors.load_word2vec_format(
        path_resource + '/tmunlp_1.6B_WB_50dim_2020v1.bin.gz', 
        unicode_errors='ignore',
        binary=True
    )

In [51]:
# model.vocab

In [50]:
model.get_vector('還')

array([-10.912628  ,   5.5556545 ,  -1.9609258 ,  -2.2676349 ,
        -2.6603632 ,   8.803495  ,   2.550427  , -15.03157   ,
        -1.2258326 ,   2.7375453 , -10.716909  ,   3.588066  ,
        -5.7324567 ,  -6.5745616 ,  11.750382  ,  12.53284   ,
        -1.6244458 ,  -6.9417815 ,   2.008579  ,  -8.293338  ,
       -10.414997  ,   2.8907945 ,   5.348273  ,  11.527352  ,
         3.8111753 ,  -9.892887  ,  -4.8136654 , -17.782816  ,
        -4.580268  ,  -9.910597  , -17.388525  , -13.203818  ,
       -19.560038  ,   4.3322673 ,  -1.4625031 ,   7.5769176 ,
        -5.5711575 ,   1.2358564 ,   1.0384434 ,  -5.7642326 ,
         5.852718  ,  -2.639902  ,  -1.9903792 ,  18.69575   ,
         2.2073245 ,  -2.0173435 ,   0.23429355,   5.508139  ,
        -3.6236613 ,  -4.839179  ], dtype=float32)

In [72]:
for i in [0]:
# for i in range(len(word_sentence_list)):

    word_vec = {}
    for w in word_sentence_list[i]:
        try:
            word_vec[w] = model.get_vector(w)
        except:
            pass

word_vec        

{'香水': array([  4.626414  ,   1.8377794 ,   5.0118885 ,  -2.7468445 ,
         -2.7426767 ,   7.874553  ,   1.6198485 , -13.614906  ,
         -2.2639399 ,   1.2704594 ,  -3.4802852 ,  -5.7270465 ,
          2.834693  ,  -9.493852  ,  -4.7917824 ,  -9.890029  ,
          0.06897594,   4.4461985 ,   2.9312348 ,  10.678728  ,
          0.8725657 ,   3.9378026 , -11.721785  ,   2.4367194 ,
         -1.9657862 ,   3.4773731 ,  -4.303724  ,  -2.9770093 ,
          1.7803626 ,  -6.7613225 ,  -2.8269513 ,  -0.17051369,
         -2.5308158 ,  -3.3007243 ,  -4.512962  ,   1.3217808 ,
         -2.0046532 , -17.194857  ,   9.740687  ,   8.009914  ,
         -4.2333064 ,  -9.278232  ,  -1.4423128 ,  -4.1769443 ,
         -1.7150015 ,   2.597849  , -13.8694315 ,   5.2736087 ,
         -2.0802748 ,  -0.97421753], dtype=float32),
 '美妝': array([  4.956061  ,  -6.369097  ,  12.553097  ,   0.09311961,
        -14.178418  ,  -1.127809  ,   5.7696247 ,  -3.1256685 ,
         -9.745538  ,   3.277206  ,   2

In [62]:

np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))

0.09958271

In [43]:
type(model)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [73]:
# model.vectors
# model.vocab

array([[-2.4727688 ,  8.788344  , -3.2833579 , ...,  0.03281746,
        -8.01406   , -0.02968044],
       [-4.6349583 ,  8.328626  , -1.075075  , ..., -3.4139404 ,
        -0.5045959 ,  5.89484   ],
       [-4.245946  ,  8.922086  ,  1.3067998 , ..., -4.7974987 ,
        -5.482261  ,  5.5655594 ],
       ...,
       [-0.2069984 , -0.8096438 , -0.21803781, ..., -1.1348562 ,
         0.35619432,  0.02554984],
       [-0.77811223, -0.5388475 ,  0.1717956 , ..., -0.33595872,
         0.01547962,  1.5065216 ],
       [-0.556597  , -0.32617614,  1.7839069 , ...,  0.45378953,
         0.86942905,  0.05036779]], dtype=float32)

In [74]:
len(model.vocab)

1723175

## Worklist

In [ ]:
# - 跟淡香水之間的距離
#   和男性、女性、中性的距離
# - 如果title中同時出現「淡香水」、「淡香精」就排除